In [1]:
import face_recognition
import cv2
import numpy as np

In [2]:
me = face_recognition.load_image_file('/Users/nawjux/Pictures/My dumb face/profile_.jpg')
me2 = face_recognition.load_image_file('/Users/nawjux/Pictures/My dumb face/me.jpg')

In [3]:
encoding_1 = face_recognition.face_encodings(me)[0]

encoding_2 = face_recognition.face_encodings(me2)[0]

In [ ]:
face_recognition.face_distance()

In [9]:
results = face_recognition.compare_faces([encoding_1], encoding_2, tolerance=.50)

In [10]:
results

[True]

In [4]:
marvel_cast = face_recognition.load_image_file('/Users/nawjux/Desktop/main_comiccon_.jpg')
face_locations = face_recognition.face_locations(marvel_cast)

In [5]:
face_encodings = face_recognition.face_encodings(marvel_cast, face_locations)

In [6]:
closest = face_recognition.face_distance(face_encodings, encoding_1)

In [7]:
closest

array([0.85415439, 0.9514891 , 0.75442028, 0.90969745, 0.89249144,
       0.96384076, 0.86490566, 0.83178913, 0.93773674, 1.03532   ,
       0.96687481, 0.85952577, 0.89908559, 0.9318554 , 0.95032461,
       0.88678967, 0.78132515, 0.95336344, 0.76727692])

In [13]:
matches = face_recognition.compare_faces(face_encodings, encoding_1, tolerance=.74, )

In [ ]:
face_recognition?

In [14]:
matches

[False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False]

In [15]:
for id, (top, right, bottom, left) in enumerate(face_locations):

    # Draw a box around the face
    cv2.rectangle(marvel_cast, (left, top), (right, bottom), (0, 0, 255), 2)

    # Draw a label with a name below the face
    cv2.rectangle(marvel_cast, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
    
    font = cv2.FONT_HERSHEY_COMPLEX_SMALL
    
    cv2.putText(marvel_cast, str(id), (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)
    
# Display the resulting image
cv2.imwrite('/Users/nawjux/Desktop/labeled.jpg', marvel_cast)

True

In [16]:
from PIL import Image, ImageDraw

face_landmarks_list = face_recognition.face_landmarks(me)

In [ ]:
video_capture = cv2.VideoCapture(0)

# Create arrays of known face encodings and their names
known_face_encodings = [
    encoding_1,

]
known_face_names = [
    "John",
]

# Initialize some variables
face_locations = []
face_encodings = []
face_names = []
process_this_frame = True

while True:
    # Grab a single frame of video
    ret, frame = video_capture.read()

    # Resize frame of video to 1/4 size for faster face recognition processing
    small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)

    # Convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
    rgb_small_frame = small_frame[:, :, ::-1]

    # Only process every other frame of video to save time
    if process_this_frame:
        # Find all the faces and face encodings in the current frame of video
        face_locations = face_recognition.face_locations(rgb_small_frame)
        face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)

        face_names = []
        for face_encoding in face_encodings:
            # See if the face is a match for the known face(s)
            matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
            name = "Unknown"

            # # If a match was found in known_face_encodings, just use the first one.
            # if True in matches:
            #     first_match_index = matches.index(True)
            #     name = known_face_names[first_match_index]

            # Or instead, use the known face with the smallest distance to the new face
            face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
            best_match_index = np.argmin(face_distances)
            if matches[best_match_index]:
                name = known_face_names[best_match_index]

            face_names.append(name)

    process_this_frame = not process_this_frame


    # Display the results
    for (top, right, bottom, left), name in zip(face_locations, face_names):
        # Scale back up face locations since the frame we detected in was scaled to 1/4 size
        top *= 4
        right *= 4
        bottom *= 4
        left *= 4

        # Draw a box around the face
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

        # Draw a label with a name below the face
        cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)
        

    # Display the resulting image
    cv2.imshow('Video', frame)

    # Hit 'q' on the keyboard to quit!
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release handle to the webcam
video_capture.release()
cv2.destroyAllWindows()